# PRiAD - Projekt zespołowy "Klasyfikacja wniosków kredytowych"

---

In [144]:
# Import potrzebnych bibliotek
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Zmiana sposobu wyświetlania danych zmiennoprzecinkowych
pd.options.display.float_format = "{:.2f}".format 

In [145]:
# Przydatne funkcje

def setProperColumnNames(data):
    data = data.rename(
        columns = {
            0:'A1', 1:'A2', 2:'A3', 3:'A4', 4:'A5', 5:'A6', 6:'A7', 7:'A8', 8:'A9',
            9:'A10', 10:'A11', 11:'A12', 12:'A13', 13:'A14', 14:'A15', 15:'A16'
        }
    )
    
    return data

def getErrorMatrix(data, testResult):
    return pd.crosstab(data['decisiveSetToTest'], testResult) 


def getTotalErrors(errorMatrix):
    totalErrors = 0

    # Błędy nad diagonalą
    for row in range(0, errorMatrix.shape[0]):
        for column in range(row + 1, errorMatrix.shape[1]):
            totalErrors += errorMatrix.iloc[row, column]

    # Błędy pod diagonalą
    for column in range(0, errorMatrix.shape[1]):
        for row in range(column + 1, errorMatrix.shape[0]):
            totalErrors += errorMatrix.iloc[row, column]

    return totalErrors

def testClassifier(classifier, relevantAttributes, data):
    sum = 0
    for i in range(10):
        dividedData = divideData(data)
        relevantTestResult = classifier.predict(
            dividedData['descriptiveSetToTest'].iloc[:, relevantAttributes]
        )
        accuracy = accuracy_score(dividedData['decisiveSetToTest'], relevantTestResult)
        sum += accuracy
    
    return sum/10

# 1. Przygotowanie danych.

Wczytanie i przygotowanie ramki danych do pracy:

In [146]:
# Konwersja .data --> .csv do typu Data Frame
data = pd.read_csv('crx.data', header = None, sep = ',')
data = setProperColumnNames(data)

# Porzucenie wierszy z podziurawionymi danymi
data = data.dropna()
for column in data.columns:
    data = data[data[column] != '?']

data

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.00,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.46,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.50,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.54,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.62,u,g,w,v,1.71,t,f,0,f,s,00120,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.09,y,p,e,h,1.25,f,f,0,f,g,00260,0,-
686,a,22.67,0.75,u,g,c,v,2.00,f,t,2,t,g,00200,394,-
687,a,25.25,13.50,y,p,ff,ff,2.00,f,t,1,t,g,00200,1,-
688,b,17.92,0.20,u,g,aa,v,0.04,f,f,0,f,g,00280,750,-


Aby otrzymać wiarygodne wyniki podczas pracy nad danymi musimy je ustandaryzować, a do tego musieliśmy je odpowiednio zakodować. Zamieniliśmy znaki i łańcuchy znaków w danych na wartości liczbowe. Następnie ustandaryzowaliśmy nasze dane, aby nie zatruć klasyfikatora przez duże różnice w zakresach przyjmowanych wartości poszczególnych atrybutów:

In [147]:
# Zakodowanie danych
columnsToEncodeIndexes = [1, 4, 5, 6, 7, 9, 10, 12, 13, 16]
for index in columnsToEncodeIndexes:
    data['A' + str(index)] = data['A' + str(index)].astype('category').cat.codes

data

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1,30.83,0.00,1,0,12,7,1.25,1,1,1,0,0,00202,0,0
1,0,58.67,4.46,1,0,10,3,3.04,1,1,6,0,0,00043,560,0
2,0,24.50,0.50,1,0,10,3,1.50,1,0,0,0,0,00280,824,0
3,1,27.83,1.54,1,0,12,7,3.75,1,1,5,1,0,00100,3,0
4,1,20.17,5.62,1,0,12,7,1.71,1,0,0,0,2,00120,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,1,21.08,10.09,2,2,4,3,1.25,0,0,0,0,0,00260,0,1
686,0,22.67,0.75,1,0,1,7,2.00,0,1,2,1,0,00200,394,1
687,0,25.25,13.50,2,2,5,2,2.00,0,1,1,1,0,00200,1,1
688,1,17.92,0.20,1,0,0,7,0.04,0,0,0,0,0,00280,750,1


In [148]:
dataBeforeStand = data

# Standaryzacja danych
sc = StandardScaler()

data = sc.fit_transform(data)
data = pd.DataFrame(data)  
data = setProperColumnNames(data)

data

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,0.67,-0.06,-0.96,-0.54,-0.55,1.45,0.77,-0.30,0.93,1.13,-0.30,-0.93,-0.30,0.13,-0.19,-1.10
1,-1.49,2.30,-0.07,-0.54,-0.55,0.98,-0.84,0.24,0.93,1.13,0.70,-0.93,-0.30,-0.82,-0.09,-1.10
2,-1.49,-0.59,-0.86,-0.54,-0.55,0.98,-0.84,-0.22,0.93,-0.89,-0.50,-0.93,-0.30,0.59,-0.04,-1.10
3,0.67,-0.31,-0.65,-0.54,-0.55,1.45,0.77,0.45,0.93,1.13,0.50,1.08,-0.30,-0.48,-0.19,-1.10
4,0.67,-0.96,0.16,-0.54,-0.55,1.45,0.77,-0.16,0.93,-0.89,-0.50,-0.93,3.35,-0.36,-0.19,-1.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,0.67,-0.88,1.05,1.80,1.81,-0.41,-0.84,-0.30,-1.07,-0.89,-0.50,-0.93,-0.30,0.47,-0.19,0.91
649,-1.49,-0.75,-0.81,-0.54,-0.55,-1.11,0.77,-0.07,-1.07,1.13,-0.10,1.08,-0.30,0.12,-0.12,0.91
650,-1.49,-0.53,1.73,1.80,1.81,-0.18,-1.24,-0.07,-1.07,1.13,-0.30,1.08,-0.30,0.12,-0.19,0.91
651,0.67,-1.15,-0.92,-0.54,-0.55,-1.34,0.77,-0.65,-1.07,-0.89,-0.50,-0.93,-0.30,0.59,-0.05,0.91


# 3. Funkcja utworzenia zbioru uczącego i testowego.

Za pomocą tej funkcji podzielimy otrzymany zbiór danych wniosków kredytowych na zbiór testowy oraz uczący tak, aby zbiór uczący był dwa razy większy od testowego. Dzięki takiej proporcji otrzymamy najbardziej wiarygodny wynik wydajności budowanego klasyfikatora:

In [149]:
# Podział na zbiór testowy/uczący po atrybutach opisowych/decyzyjnych:

def divideData(data, proportion = 0.33):
    descriptiveSetToTeach, descriptiveSetToTest, decisiveSetToTeach, decisiveSetToTest = train_test_split(
        data.iloc[:,0:-1], 
        data.iloc[:,-1].astype('category').cat.codes, 
        test_size = proportion
    )

    dividedData = {
        'descriptiveSetToTeach' : descriptiveSetToTeach, 
        'descriptiveSetToTest' : descriptiveSetToTest, 
        'decisiveSetToTeach' : decisiveSetToTeach, 
        'decisiveSetToTest' : decisiveSetToTest
    }

    return dividedData


# 4. Próbna budowa klasyfikatora i klasyfikacja.

W ramach naszego klasyfikatora przetestowaliśmy poniższe:
* K Nearest Neighbors
* Nearest Centroid
* Gaussian Naive Bayes
* Random Forest Classifier 

Klasyfikator k najbliższych sąsiadów okazał się najskuteczniejszy, ponieważ:
* Osiągał największy wynik procentowy poprawnie zklasyfikowanych wnioskow kredytowych
* Z innych klasyfikatorów o podobnych wynikach (Najbliższego Centroidu) w przeciągu kilkunastu uruchomień wynik procentowy poprawnie zklasyfikowanych wniosków kredytowych wahał się najmniej

Dla wybranego klasyfikatora zastanowiliśmy się nad najlepszymi możliwymi jego parametrami. 
* Sprawdziliśmy kolejne nieparzyste wartości k = 3, 5, 7, 9, 11. Najlepszym i najbardziej stabilnym wynikiem (na tle kilkunastu wywołań) cieszyło się k = 5.  
* Po próbach testów kolejno przyjętych metryk odległości (euklidesowa, manhattan, chevyshev, minkowski etc.) najlepszą okazała się 'minkowski'. 
* Podobnie postąpiliśmy z algorytmami - najlepsze (na podobnym poziomie) okazały się algorytmy 'ball_tree' oraz 'brute', więc na podstawie złożoności czasowej wybraliśmy 'ball_tree' jako ten bardziej optymalny.

In [150]:
# Zbudowanie klasyfikatora
classifier = KNeighborsClassifier(
    n_neighbors = 5, 
    algorithm = 'ball_tree', 
    metric = 'minkowski'
)

classifier.fit(
    dividedData['descriptiveSetToTeach'], 
    dividedData['decisiveSetToTeach']
)


KNeighborsClassifier(algorithm='ball_tree')

Po zbudowaniu klasyfikatora wykonujemy klasyfikację próbną, a następnie wyświetlamy jej wynik w postaci macierzy błędów oraz procentu poprawnie zaklasyfikowanych wektorów. Na podstawie tego będziemy w stanie ocenić skuteczność zbudowanego przez nas klasyfikatora.

In [151]:
# Wykonanie testu klasyfikacji
descriptiveAttributes = range(0, data.columns.size - 1)
averageResult = testClassifier(classifier, descriptiveAttributes, data)


In [152]:
# Wyświetlenie wyniku pracy klasyfikatora
print('Average accuracy for all atributes: ', averageResult)


Average accuracy for all atributes:  0.8736111111111112


Skuteczność na poziomie 85% wydaje się być poprawna, ale w tak wrażliwym obszarze jakim jest bankowość jest zdecydowanie zbyt niska. Aby podnieść skuteczność klasyfikacji danych przeprowadzimy analizę eksploracyjną, która wyłoni czy poszczególne pojedyncze atrybuty są w stanie wskazać wynik klasyfikacji bardziej niż pozostałe.

# 5. Analiza eksploracyjna danych.

Histogramy dla poszczególnych atrybutów:

In [154]:
# Tworzenie histogramu na podstawie danych sprzed standaryzacji, ale po kodowaniu
feature_dict = {i:label for i,label in zip(
                range(15), dataBeforeStand.columns)}

label_dict = {0: 'Przyjęty', 1: 'Odrzucony'}
import math

fig, axes = plt.subplots(nrows=8, ncols=2, figsize=(12,16))

for ax,cnt in zip(axes.ravel(), range(15)):
    
    min_b = math.floor(np.min(dataBeforeStand.iloc[:, :-1].values[:,cnt]))
    max_b = math.ceil(np.max(dataBeforeStand.iloc[:, :-1].values[:,cnt]))
    bins = np.linspace(min_b, max_b, 25)

    # Tworzenie kolorowego histogramu (kolor zielony - zatwierdzone, czerwony - odrzucone,
    # brązowe - nakładające się wartości, tzn. zatwierdzone wnioski i odrzucone się na siebie nakładają)
    for lab,col in zip(range(0,2), ('green', 'red')):
        ax.hist(dataBeforeStand.iloc[:, :-1].values[dataBeforeStand.iloc[:, -1].values==lab, cnt],
                   color=col,
                   label='Wniosek %s' %label_dict[lab],
                   bins=bins,
                   alpha=0.5,)
    ylims = ax.get_ylim()

    # Adnotacje wykresu
    leg = ax.legend(loc='upper right', fancybox=True, fontsize=8)
    leg.get_frame().set_alpha(0.5)
    ax.set_ylim([0, max(ylims)+2])
    ax.set_xlabel(feature_dict[cnt])
    ax.set_title('Atrybut A%s' %str(cnt+1))

    
fig.tight_layout()
plt.show()


TypeError: must be real number, not str

Przeprowadźmy badanie korelacji poszczególnych atrybutów, z których być może uda się wyłonić atrybuty nadmiarowe:

In [ ]:
# Budowa macierzy korelacji atrybutów
dataCorr = data.corr()

# Wyświetlenie macierzy korelacji
plt.figure(figsize = (15,15), dpi = 100)
sns.heatmap(dataCorr, annot = dataCorr)

Z przeprowadzonej analizy macierzy korelacji pomiędzy atrybutami możemy wysunąć następujące wnioski:
* Najbardziej silne skorelowane ze sobą pary atrybutów:
    * {A4, A5}     -> 0.98
    * {A9, A16}    -> - 0,74
    * {A10, A11}   -> 0.57
* Nie oznacza to wcale logicznego z punktu biznesowego powiązania pomiędzy tymi atrybutami w obie strony. Najczęściej to jeden atrybut z każdej pary definiuje drugi, ale drugi wcale nie definiuje pierwszego. Problem polega na prawidłowej ocenie, którego nadmiarowego atrybutu możemy się pozbyć z procesu kwalifikacji. 
* Ponieważ tytuły atrybutów zostały zakodowanego w celu zachowania poufności ich prawdziwej tożsamości nie jesteśmy w stanie stwierdzić, który jest skutkiem którego. 
* Z racji bardzo wysokiej korelacji atrybutów A4 i A5 możemy w przybliżeniu założyć, że są tak bardzo zależne od siebie, że nie ma aż takiego znaczeni, który jest skutkiem którego. Oznacza to, że możemy pozbyć się jednego z nich bez potrzeby znajomości jego prawdziwej tożsamości. Decyzję podejmiemy na podstawie wartości korelacji obu atrybutów z pozostałymi atrybutami danych i pozbędziemy się tego, którego korelacje przyjmują największe wartości. 
    * Po zakończeniu analizy zakładamy, że będziemy mogli pozbyć się atrybutu **A4** i poprawi to wynik klasyfikacji.
* Rzędy dla A16 mogą okazać się kluczowe dla naszej analizy. Atrybutem decyzyjny A16 jest silnie skorelowany z atrybutem A9, który przyjmuje wartości t/f. Dla 74% przypadków, jeżeli A9 przyjęło wartość 'true' wyniosek został przyjęty. 
    * Z racji tego **A9** jest bardzo ważnym atrybutem.


Narysujmy teraz wykresy przedstawiające wynik klasyfikacji w zakresie przyjmowanych wartości poszczególnych atrybutów:

In [ ]:
# Histograms for each of the columns

plt.figure(figsize = (20, 15))
for i in range(15):
    plt.subplot(5, 3, i + 1)
    j = str(1 + i)
    sns.violinplot(x = 'A16', y = 'A' + j, data = data)

Ważne atrybuty przy kwalifikacji:
* A8
* A9
* A10

Atrybuty o umiarkowanym znaczeniu przy klasyfikacji:
* A3
* A4
* A6
* A11
* A14
* A15

Nieważne atrybuty przy kwalifikacji:
* A1
* A2
* A5
* A7
* A12
* A13

In [ ]:
# Listę wypełniamy indeksami kolumn zaczynając od 0, czyli np. dla A6 indeks na liście będzie równy 5.
relevantAttributes = [2, 3, 5, 7, 8, 9, 10, 14]

# Budowa macierzy korelacji atrybutów
dataCorr = data.iloc[:, relevantAttributes].corr()

# Wyświetlenie macierzy korelacji
plt.figure(figsize = (10,10), dpi = 100)
sns.heatmap(dataCorr, annot = dataCorr)

Wnioski:
* A11 i A10 są silnie skorelowane ze sobą. Odrzucamy jeden atrybut nadmiarowy. Na podstawie analizy przydatności obu atrybutów odrzucamy A11 

In [ ]:
relevantAttributes.remove(10)

# 6. Ponowna budowa klasyfikatora i klasyfikacja na podstawie wniosków z analizy eksploracyjnej.

Uwzględniając wyłonione podczas analizy najważniejsze atrybuty zbudujemy ponownie klasyfikator na podstawie przyjętych atrybutów i od nowa przeprowadźmy klasyfikację zbioru testowego:

In [ ]:
# Zbudowanie klasyfikatora
relevantClassifier = KNeighborsClassifier(
    n_neighbors = 5, 
    algorithm = 'ball_tree', 
    metric = 'minkowski'
)

relevantClassifier.fit(
    dividedData['descriptiveSetToTeach'].iloc[:, relevantAttributes], 
    dividedData['decisiveSetToTeach']
)


Ponowne wykonanie klasyfikacji i wyświetlenie jej wyniku w postaci macierzy błędów oraz procentu poprawnie zaklasyfikowanych wektorów. Na podstawie tego będziemy w stanie ocenić skuteczność zbudowanego przez nas klasyfikatora i przeprowadzonej analizy eksploracyjnej:

In [ ]:
# Wykonanie klasyfikacji
averageResultForRelevant = testClassifier(relevantClassifier, relevantAttributes, data)


In [ ]:
# Wyświetlenie wyniku pracy klasyfikatora
print('Average accuracy for just relevant atributes: ', averageResult)

OCENA WYNIKU KLASYFIKACJI

# 7. Podsumowanie.

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.